<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [5]:
print(response.status_code)

200


In [8]:
#ANSWER
print(response.text)

{"message": "success", "timestamp": 1695196001, "iss_position": {"longitude": "-66.2572", "latitude": "-14.8117"}}


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [9]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [10]:
#ANSWER
print(response.headers)

{'Date': 'Wed, 20 Sep 2023 07:50:20 GMT', 'Server': 'Apache', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [12]:
#ANSWER
print(response.content)

b'{"api_status":"ALPHA","request_timestamp":"2023-09-20T07:50:20.388371705Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2023-09-19T14:06:21.787220224Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":19,"passes":[{"start":"2023-09-20T17:03:30.369Z","tca":"2023-09-20T17:08:00.369Z","end":"2023-09-20T17:12:30.369Z","aos_azimuth":359,"los_azimuth":109,"max_elevation":11.0},{"start":"2023-09-20T18:38:55.369Z","tca":"2023-09-20T18:44:25.369Z","end":"2023-09-20T18:49:55.369Z","aos_azimuth":304,"los_azimuth":136,"max_elevation":54.0},{"start":"2023-09-20T20:17:25.369Z","tca":"2023-09-20T20:21:55.369Z","end":"2023-09-20T20:26:10.369Z","aos_azimuth":255,"los_azimuth":151,"max_elevation":9.0},{"start":"2023-09-20T21:57:15.369Z","tca":"2023-09-20T22:00:15.369Z","end":"2023-09-20T22:03:05.369Z","aos_azimuth":214,"los_azimuth":150,"max_elevation":3.0},{"start":"2023-09-20T23:34:50.369Z","tca":"2023-09-20T23:38:50.369Z","end":"2023-09-20T23:42:30.369Z","aos_azimu

Note that this is a Python byte string:

In [ ]:
print(type(response.content))

Print just the "content-type" value from the header:

In [14]:
#ANSWER
print(type(response.content))

<class 'bytes'>


In [16]:
print(response.headers["content-type"])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [17]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2023-09-20T07:50:20.388371705Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2023-09-19T14:06:21.787220224Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 19, 'passes': [{'start': '2023-09-20T17:03:30.369Z', 'tca': '2023-09-20T17:08:00.369Z', 'end': '2023-09-20T17:12:30.369Z', 'aos_azimuth': 359, 'los_azimuth': 109, 'max_elevation': 11.0}, {'start': '2023-09-20T18:38:55.369Z', 'tca': '2023-09-20T18:44:25.369Z', 'end': '2023-09-20T18:49:55.369Z', 'aos_azimuth': 304, 'los_azimuth': 136, 'max_elevation': 54.0}, {'start': '2023-09-20T20:17:25.369Z', 'tca': '2023-09-20T20:21:55.369Z', 'end': '2023-09-20T20:26:10.369Z', 'aos_azimuth': 255, 'los_azimuth': 151, 'max_elevation': 9.0}, {'start': '2023-09-20T21:57:15.369Z', 'tca': '2023-09-20T22:00:15.369Z', 'end': '2023-09-20T22:03:05.369Z', 'aos_azimuth': 214, 'los_azimuth': 150, 'max_elevation': 3.0}, {'start': '2023-09-20T23:34:50.369Z', 'tca': 

What kind of object did this give us?

In [18]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate list:

In [19]:
#ANSWER:
passes = overheads['passes']


Now extract the `start` strings into an array called `srisetimes`:

In [ ]:
#ANSWER:


These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [ ]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [ ]:
#ANSWER:


Finally, here is an endpoint that tells us who is on board:

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [ ]:
#ANSWER:


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [ ]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

>

>

>



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



